# Multi-Agent Experiments: Debate Strategy

**Experiment:** Compare debate-based multi-agent vs single-model baselines  
**Date:** [Fill in]  
**Author:** Leif Haven Martinson  

## Goals
- Implement 2-agent debate with judge
- Compare debate vs single-model baseline
- Measure quality, latency, and cost tradeoffs
- Identify when debate helps vs hurts

## Hypotheses
- Debate improves accuracy on tasks with multiple valid perspectives
- Debate adds 2-3x latency but may justify cost with quality gains
- Judge quality matters more than debater quality


In [1]:
# Setup
import sys
sys.path.append('../code')

from harness import (
    llm_call,
    run_strategy,
    debate_strategy,
    ExperimentConfig,
    ExperimentResult,
    get_tracker,
    evaluate_task
)

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_style('whitegrid')

print("✅ Setup complete")

Matplotlib is building the font cache; this may take a moment.


✅ Setup complete


## Debate Strategy Implementation

Two agents debate, then a judge decides the best answer.

In [2]:
# The harness already has debate strategy built-in!
# Let's test it with a simple example

test_result = run_strategy(
    "debate",
    "What is 2+2?",
    n_debaters=2,
    provider="ollama",
    model="gpt-oss:20b"
)

print(f"Debate result: {test_result.output}")
print(f"Latency: {test_result.latency_s:.2f}s")
print(f"\nNumber of debaters: {test_result.metadata['n_debaters']}")
print("\nDebate system ready!")

Debate result: Answer 1 is best because it not only gives the correct result (4) but also explains why that result is correct, referencing standard arithmetic and providing a brief justification. This additional context makes the answer clearer and more informative than the terse statement in Answer 2.
Latency: 33.73s

Number of debaters: 2

Debate system ready!


## Load Tasks from Baseline

Use the same tasks for fair comparison.

In [3]:
# Same tasks as baseline experiment
reasoning_tasks = [
    {
        "id": "logic_01",
        "category": "logical_reasoning",
        "input": "If all roses are flowers and some flowers fade quickly, can we conclude that some roses fade quickly?",
        "expected": "No, this doesn't follow logically."
    },
    {
        "id": "math_01",
        "category": "arithmetic",
        "input": "A train travels 120 km in 2 hours, then 180 km in 3 hours. What is its average speed for the entire journey?",
        "expected": "60 km/h"
    },
    {
        "id": "reasoning_01",
        "category": "causal_reasoning",
        "input": "Studies show that people who drink coffee tend to live longer. Does this mean coffee causes longevity?",
        "expected": "No, correlation doesn't imply causation."
    },
    {
        "id": "planning_01",
        "category": "planning",
        "input": "You need to be at a meeting 30 km away at 2 PM. Traffic is heavy (20 km/h). It's now 1:15 PM. Can you make it on time?",
        "expected": "No. Travel time = 1.5 hours."
    },
    {
        "id": "pattern_01",
        "category": "pattern_recognition",
        "input": "What comes next in this sequence: 2, 6, 12, 20, 30, ?",
        "expected": "42"
    }
]

print(f"Loaded {len(reasoning_tasks)} tasks")

Loaded 5 tasks


## Configure Debate Models

Start with same model for debaters and judge.

In [4]:
# Configuration
DEBATER_MODEL = "gpt-oss:20b"  # Using Ollama for speed
JUDGE_MODEL = "gpt-oss:20b"     # Can use larger model later
NUM_DEBATERS = 2
PROVIDER = "ollama"

print(f"Debate configuration: {NUM_DEBATERS} debaters + 1 judge")
print(f"Provider: {PROVIDER}")
print(f"Model: {DEBATER_MODEL}")

Debate configuration: 2 debaters + 1 judge
Provider: ollama
Model: gpt-oss:20b


## Run Debate Experiments

In [5]:
# Run debate experiment with tracking
config = ExperimentConfig(
    experiment_name=f"debate_{NUM_DEBATERS}agents",
    task_type="reasoning",
    strategy="debate",
    provider=PROVIDER,
    model=DEBATER_MODEL,
    n_agents=NUM_DEBATERS,
    notes=f"{NUM_DEBATERS} debaters with judge"
)

tracker = get_tracker()
tracker.start_experiment(config)

for i, task in enumerate(reasoning_tasks):
    print(f"\n{'='*60}")
    print(f"Running debate on: {task['id']}")
    print(f"{'='*60}")
    
    # Run debate strategy
    result = run_strategy(
        "debate",
        task['input'],
        n_debaters=NUM_DEBATERS,
        provider=PROVIDER,
        model=DEBATER_MODEL
    )
    
    # Display arguments
    for j, arg in enumerate(result.metadata['arguments']):
        print(f"\nDebater {j+1}: {arg[:100]}...")
    
    print(f"\nJudge verdict: {result.output[:100]}...")
    print(f"Total latency: {result.latency_s:.2f}s")
    
    # Log result
    exp_result = ExperimentResult(
        config=config,
        task_input=task['input'],
        output=result.output,
        latency_s=result.latency_s,
        tokens_in=result.tokens_in,
        tokens_out=result.tokens_out,
        cost_usd=result.cost_usd,
        eval_metadata={
            'task_id': task['id'],
            'category': task['category'],
            'expected': task['expected'],
            'arguments': result.metadata['arguments']
        }
    )
    
    # Evaluate the result
    exp_result.eval_scores = evaluate_task(task, result.output)
    
    tracker.log_result(exp_result)
    print("✓ Logged")

summary = tracker.finish_experiment()
print("\n" + "="*60)
print("Debate experiment complete!")
print(f"Saved to: {tracker.current_run_dir}")

📊 Started experiment: debate_2agents_20251026_141040_0e0a6d97
📁 Logging to: experiments/debate_2agents_20251026_141040_0e0a6d97

Running debate on: logic_01

Debater 1: **Answer (Debater 1)**  

No. The two premises do not allow us to conclude that any roses fade quick...

Debater 2: **No – the inference is invalid.**

1. **Formal structure**  
   - Premise 1: All roses are flowers....

Judge verdict: **Answer 1 is best because** it fully explains the logical structure, shows why the inference fails,...
Total latency: 149.42s


ResponseError: model 'llama3.2:latest' not found (status code: 404)

## Compare: Single vs Debate

Load baseline and debate results for comparison.

In [ ]:
# Load and compare experiments
# Note: You'll need to run the baseline experiment first (01_baseline_experiments.ipynb)
# Then update the paths below with your actual experiment directories

import os
from pathlib import Path

# List available experiments
exp_dir = Path("../experiments")
if exp_dir.exists():
    experiments = sorted([d.name for d in exp_dir.iterdir() if d.is_dir()])
    print("Available experiments:")
    for exp in experiments:
        print(f"  - {exp}")
else:
    print("No experiments found yet. Run baseline experiments first!")

In [ ]:
# Comparison metrics
comparison_df = pd.DataFrame([
    {
        'Strategy': 'Single Model',
        'Avg Latency (ms)': baseline_data['summary']['stats']['avg_latency_ms'],
        'Total Cost ($)': baseline_data['summary']['stats']['total_cost'],
        'Runs': baseline_data['summary']['num_runs']
    },
    {
        'Strategy': f'Debate ({NUM_DEBATERS} agents)',
        'Avg Latency (ms)': debate_data['summary']['stats']['avg_latency_ms'],
        'Total Cost ($)': debate_data['summary']['stats']['total_cost'],
        'Runs': debate_data['summary']['num_runs']
    }
])

# Calculate overhead
latency_overhead = (debate_data['summary']['stats']['avg_latency_ms'] / 
                   baseline_data['summary']['stats']['avg_latency_ms'])
cost_overhead = (debate_data['summary']['stats']['total_cost'] / 
                baseline_data['summary']['stats']['total_cost'])

print("\nPerformance Comparison:")
print(comparison_df.to_string(index=False))
print(f"\nDebate Overhead:")
print(f"  Latency: {latency_overhead:.1f}x slower")
print(f"  Cost: {cost_overhead:.1f}x more expensive")

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Latency comparison
ax = axes[0]
comparison_df.plot.bar(x='Strategy', y='Avg Latency (ms)', ax=ax, legend=False, color=['steelblue', 'coral'])
ax.set_title('Latency: Single vs Debate', fontsize=14, weight='bold')
ax.set_ylabel('Milliseconds')
ax.set_xlabel('')
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')

# Cost comparison
ax = axes[1]
comparison_df.plot.bar(x='Strategy', y='Total Cost ($)', ax=ax, legend=False, color=['steelblue', 'coral'])
ax.set_title('Cost: Single vs Debate', fontsize=14, weight='bold')
ax.set_ylabel('USD')
ax.set_xlabel('')
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.show()

## Qualitative Analysis

Compare actual outputs to assess quality differences.

In [ ]:
def compare_outputs(task_id: str):
    """Show baseline vs debate outputs side by side."""
    print(f"\n{'='*80}")
    print(f"Task: {task_id}")
    print(f"{'='*80}\n")
    
    # Get task
    task = next(t for t in reasoning_tasks if t['id'] == task_id)
    print(f"Input: {task['input']}\n")
    print(f"Expected: {task['expected']}\n")
    print("-" * 80)
    
    # Get baseline
    baseline_run = next(r for r in baseline_data['runs'] if r['metadata']['task_id'] == task_id)
    print(f"\nBASELINE (Single Model):")
    print(f"Output: {baseline_run['output']}")
    print(f"Latency: {baseline_run['latency_ms']:.0f}ms")
    
    # Get debate
    debate_run = next(r for r in debate_data['runs'] if r['metadata']['task_id'] == task_id)
    print(f"\nDEBATE ({NUM_DEBATERS} agents):")
    for i, arg in enumerate(debate_run['metadata']['arguments']):
        print(f"  Debater {i+1}: {arg[:80]}...")
    print(f"\nJudge Verdict: {debate_run['output']}")
    print(f"Latency: {debate_run['latency_ms']:.0f}ms")
    print("-" * 80)

# Compare first task
compare_outputs(reasoning_tasks[0]['id'])

In [ ]:
# Browse all tasks
for task in reasoning_tasks:
    compare_outputs(task['id'])

## Key Findings

### Quantitative
- [Fill in after running]
- Debate adds Xx latency overhead
- Cost increased by X%

### Qualitative
- [Observations on quality differences]
- When did debate help?
- When did debate hurt?

### Hypotheses
- [ ] Debate improves accuracy on multi-perspective tasks
- [ ] Latency overhead is 2-3x
- [ ] Judge quality matters more than debaters

## Next Experiments
1. Test with larger judge model (13B or 30B)
2. Add 3rd debater
3. Try specialized debater roles
4. Test on creative/open-ended tasks
